In [1]:
import os
import sys
import pickle

import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import random
from matplotlib.patches import Patch
import scipy

from matplotlib.lines import Line2D
import matplotlib as mpl
import pandas as pd
import seaborn as sns


In [3]:

# Get the current working directory
current_dir = os.getcwd()

# Navigate to 'network_simulations'
parent_dir = os.path.abspath(os.path.join(current_dir, ".."))

# Navigate to the sibling directory '00_opinion_function_setup'
sibling_dir = os.path.join(parent_dir, "00_opinion_function_setup")

# Add the sibling directory to sys.path for importing
if sibling_dir not in sys.path:
    sys.path.append(sibling_dir)

# Import the module from 00_opinion_function_setup
import opinion_functions as fun  
import generate_homophilic_graph_symmetric

In [4]:


# Parameters
data_folder = "robustness_data"  # Adjusted relative path
homophily_values = [0.0, 0.25, 0.5, 0.75, 1]
m_values = [2, 5]
num_agents_values = [1000]  # Use the correct number of agents
num_sim = 1000

simulation_results = fun.load_simulation_results_with_swaps(data_folder, homophily_values, m_values, num_agents_values, num_sim)


Files in output folder:
homophily_0.0_m_2_num_agents_1000_sim_1000_maj.pkl
homophily_0.0_m_2_num_agents_1000_sim_1000_maj_swapped.pkl
homophily_0.0_m_2_num_agents_1000_sim_1000_min.pkl
homophily_0.0_m_2_num_agents_1000_sim_1000_min_swapped.pkl
homophily_0.0_m_5_num_agents_1000_sim_1000_maj.pkl
homophily_0.0_m_5_num_agents_1000_sim_1000_maj_swapped.pkl
homophily_0.0_m_5_num_agents_1000_sim_1000_min.pkl
homophily_0.0_m_5_num_agents_1000_sim_1000_min_swapped.pkl
homophily_0.25_m_2_num_agents_1000_sim_1000_maj.pkl
homophily_0.25_m_2_num_agents_1000_sim_1000_maj_swapped.pkl
homophily_0.25_m_2_num_agents_1000_sim_1000_min.pkl
homophily_0.25_m_2_num_agents_1000_sim_1000_min_swapped.pkl
homophily_0.25_m_5_num_agents_1000_sim_1000_maj.pkl
homophily_0.25_m_5_num_agents_1000_sim_1000_maj_swapped.pkl
homophily_0.25_m_5_num_agents_1000_sim_1000_min.pkl
homophily_0.25_m_5_num_agents_1000_sim_1000_min_swapped.pkl
homophily_0.5_m_2_num_agents_1000_sim_1000_maj.pkl
homophily_0.5_m_2_num_agents_1000_sim

In [5]:
# Keyed by (homophily, m, num_agents), where m=2 and num_agents=1000
m = 2
num_agents = 1000
homophilyvec = [0.0, 0.25, 0.5, 0.75, 1]
minority_fraction = 0.333
majority_fraction = 1 - minority_fraction
num_sim = 1000 # Number of simulations
# Load results

In [35]:
# Initialize dictionaries to store results and their statistics for each key
mean_majority_opinion_stats = {}
mean_minority_opinion_stats = {}
mean_overall_opinion_stats = {}

# Iterate over all keys in simulation_results
for result_key in simulation_results.keys():
    homophily, m, num_agents, swap_idx = result_key

    # Extract simulation data for the current swap
    majority_data = simulation_results[result_key]["majority"]
    minority_data = simulation_results[result_key]["minority"]

    # Compute mean opinions for the current swap
    mean_majority = np.mean(majority_data, axis=1)  # Mean for each simulation
    mean_minority = np.mean(minority_data, axis=1)  # Mean for each simulation
    mean_overall = minority_fraction * mean_minority + majority_fraction * mean_majority

    # Compute statistics for the current swap
    mean_majority_opinion_stats[result_key] = {
        "mean": np.mean(mean_majority),
        "std": np.std(mean_majority),
        "min": np.min(mean_majority),
        "max": np.max(mean_majority),
    }

    mean_minority_opinion_stats[result_key] = {
        "mean": np.mean(mean_minority),
        "std": np.std(mean_minority),
        "min": np.min(mean_minority),
        "max": np.max(mean_minority),
    }

    mean_overall_opinion_stats[result_key] = {
        "mean": np.mean(mean_overall),
        "std": np.std(mean_overall),
        "min": np.min(mean_overall),
        "max": np.max(mean_overall),
    }

# Example of accessing the statistics for a specific swap
# Example: mean_majority_opinion_stats[(homophily, m, num_agents, swap_idx)]["min"]


In [ ]:
# Specify the swaps you want to include
swaps_to_include = [10, 20, 50, 100, 200, 300]

# Helper function to add rows for each result_key
def add_to_multiindex_data(result_key, stats_dicts):
    homophily, m, num_agents, swap_idx = result_key  # Unpack the updated key

    # Ensure only keys for the current swap are included
    if swap_idx not in swaps_to_include:
        return  # Skip this result_key if it does not match the current swap
    
    row = {
        "homophily": homophily  # Add homophily to the row
    }
    # Add statistics for each opinion type
    for opinion_type, stats_dict in stats_dicts.items():
        row[(opinion_type, "mean")] = np.round(stats_dict["mean"], 2)
        row[(opinion_type, "std")] = np.round(stats_dict["std"], 2)
        row[(opinion_type, "min")] = np.round(stats_dict["min"], 2)
        row[(opinion_type, "max")] = np.round(stats_dict["max"], 2)
    multiindex_data.append(row)

# Process data and generate separate tables for each swap
for swap in swaps_to_include:
    multiindex_data = []  # Clear data for this swap
    unique_keys = list({(key[0], key[3]): key for key in mean_majority_opinion_stats.keys()}.values())
    for result_key in unique_keys:
        if result_key[3] == swap:  # Ensure swap matches
            stats_dicts = {
                "majority": mean_majority_opinion_stats[result_key],
                "minority": mean_minority_opinion_stats[result_key],
                "overall": mean_overall_opinion_stats[result_key],
            }
            add_to_multiindex_data(result_key, stats_dicts)
    
    # Create the DataFrame
    multiindex_df = pd.DataFrame(multiindex_data)
    multiindex_df = multiindex_df.set_index(['homophily'])  # Only include 'homophily' in the index

    # Set the MultiIndex columns
    multiindex_df.columns = pd.MultiIndex.from_tuples(multiindex_df.columns)

    # Construct the file name
    m_value, num_agents_value = m, num_agents  # Use appropriate values from the last processed key
    file_name = f"summary_stats/summary_statistics_m{m_value}_agents{num_agents_value}_swap_{swap}.csv"

    # Save the DataFrame to the file
    multiindex_df.to_csv(file_name)
    print(f"DataFrame saved for swap={swap}: {file_name}")


PermissionError: [Errno 13] Permission denied: 'output/summary_statistics_m5_agents1000_swap_10.csv'

In [ ]:
multiindex_df